In [1]:
#Slef-made Function
from Dataload import dataload
#from module import transformer,video_model

from module import video_model_attention
from constant import EMOTIPATH,EMOTIFACEPATH,EMOTIAUDIOPATH
from train import methods


#Torch Library

import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

#Sub tools
import pandas as pd

import os
import matplotlib.pylab as plt

#Util Library
import time
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import ImageGrid  

Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video_pt=os.path.join(EMOTIPATH,"pt","Train")
Train_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Train")
Audio_Train=os.path.join(EMOTIAUDIOPATH,"training_audio_compare.csv")

Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video_pt=os.path.join(EMOTIPATH,"pt","Valid")
Val_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Valid")
Audio_Valid=os.path.join(EMOTIAUDIOPATH,"validation_audio_compare.csv")


train_table = pd.read_csv(Train_label,delimiter=' ')
val_table = pd.read_csv(Val_labels,delimiter=' ')
print(train_table['Label'].value_counts())
print(val_table['Label'].value_counts())



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)



frame_num=10

train_data_pt=dataload.Video_Frame_Data(Train_label,base_path_v=Train_video_pt,face_path=Train_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Train)
valid_data_pt=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video_pt,face_path=Val_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Valid)





def load_pretrained_model(pre_train):
    face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_face.h")
    
   # face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_facev2.h")
   # audio_model=video_model_attention.AudioRecognition(softmax=pre_train)
   # audio_model.load_state_dict(torch.load("pre_trained_model/pre_embedded_audio.pth"),strict=False)
    image_model=video_model_attention.Video_modeller(frame_num,face_model=face_model,pre_train=pre_train)
   # image_model.load_state_dict(torch.load("saved_model_img/img_global_5.pth"),strict=False)
    return(image_model)#,audio_model)



def show_img(dataset,index,frame):
    x=dataset[index][0][frame].cpu().numpy()
    plt.figure(1,(15,15))
    plt.axis('off')
    image = (x*0.5+0.5).transpose((1, 2, 0)).squeeze()
    plt.imshow(image)
    
    fig = plt.figure(2,(15,15))
   # plt.axis('off')
    grid = ImageGrid(fig, 111,
                     nrows_ncols=(1,5),
                     axes_pad=0.1,
                     )
    for i in range(5):
        face=dataset[index][1][frame].cpu().numpy()
        image = (face[i,:]*0.5+0.5).transpose((1, 2, 0)).squeeze()
        grid[i].imshow(image,cmap='gray',interpolation='none')
        
img_model=load_pretrained_model(True)
#audio_model=video_model_attention.AudioRecognition(softmax=True)


3    932
2    923
1    806
Name: Label, dtype: int64
1    299
2    281
3    186
Name: Label, dtype: int64


In [2]:


weights=[932/932,932/923,932/806]
class_weights = torch.FloatTensor(weights).to(device)

train_dataloader = DataLoader(train_data_pt, batch_size=32
                       , num_workers=0,shuffle=True)
valid_dataloader = DataLoader(valid_data_pt, batch_size=32
                   , num_workers=0)
#model=video_model.video_transformer(img_model,audio_model)
model=img_model
model=model.to(device)

num_epochs=50
criterion = nn.CrossEntropyLoss(weight=class_weights)  

for name, child in model.named_children():
    if not name in ['face_model','frame_model' ]:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
               param.requires_grad = False

frame_model is frozen
face_model is frozen
en1 is unfrozen
en2 is unfrozen
fc1 is unfrozen
fc2 is unfrozen
fc3 is unfrozen
dropout is unfrozen


In [3]:
model(train_data_pt[6][0].unsqueeze(0).float().to(device),None)

tensor([[0.1466, 0.3838, 0.4696]], grad_fn=<SoftmaxBackward>)

In [4]:
model(None,train_data_pt[0][1].unsqueeze(0).float().to(device))

tensor([[0.1741, 0.3477, 0.4782]], grad_fn=<SoftmaxBackward>)

In [5]:

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005, betas=(0.5, 0.999))


model_name="global_attetnion"

print(model_name)

methods.frame_train("global_image_train_v2",num_epochs,model_name,model,train_dataloader,valid_dataloader,optimizer,criterion,device)


global_attetnion
Epoch:  1 

Validation 

Validation Loss:  1.122681496044
Validation Accuracy:  0.3968668407310705
Confusion Matrix: 
 [[  0 292   7]
 [  0 267  14]
 [  0 149  37]]


Batch:  40 / 84
Batch Recognition loss:  1.067987084388733
Batch:  80 / 84
Batch Recognition loss:  1.0076818466186523

Average_Loss:  0.978890468676885
Average_Accuracy:  0.5377677564825254
Confusion Matrix: 
 [[304 261 241]
 [203 480 240]
 [112 173 647]]
Epoch:  2 

Validation 

Validation Loss:  1.0212591538826625
Validation Accuracy:  0.4960835509138381
Confusion Matrix: 
 [[164  39  96]
 [140  97  44]
 [ 23  44 119]]


Batch:  40 / 84
Batch Recognition loss:  0.8937707543373108
Batch:  80 / 84
Batch Recognition loss:  1.0243604183197021

Average_Loss:  0.9255925913651785
Average_Accuracy:  0.608793686583991
Confusion Matrix: 
 [[391 228 187]
 [171 552 200]
 [ 88 167 677]]
Epoch:  3 

Validation 

Validation Loss:  1.0105154166618984
Validation Accuracy:  0.5052219321148825
Confusion Matrix: 
 [[174  55  70]
 [114 138  29]
 [ 47  64  75]]


Batch:  40 / 84
Batch Recognition loss:  0.8208170533180237
Batch:  80 / 84
Batch Recognition loss:  0.9051774740219116

Average_Loss:  0.8968622407742909
Average_Accuracy:  0.6366027809094326
Confusion Matrix: 
 [[417 218 171]
 [177 584 162]
 [ 87 152 693]]
Epoch:  4 

Validation 

Validation Loss:  0.9821802228689194
Validation Accuracy:  0.5339425587467362
Confusion Matrix: 
 [[126  70 103]
 [ 63 138  80]
 [ 20  21 145]]


Batch:  40 / 84
Batch Recognition loss:  0.8362656831741333
Batch:  80 / 84
Batch Recognition loss:  0.9653229713439941

Average_Loss:  0.876470535283997
Average_Accuracy:  0.6557685080796692
Confusion Matrix: 
 [[459 184 163]
 [173 575 175]
 [ 98 123 711]]
Epoch:  5 

Validation 

Validation Loss:  1.001938745379448
Validation Accuracy:  0.5078328981723238
Confusion Matrix: 
 [[119  76 104]
 [ 87 134  60]
 [ 28  22 136]]


...

Batch:  40 / 84
Batch Recognition loss:  0.7736694812774658
Batch:  80 / 84
Batch Recognition loss:  0.9468644261360168

Average_Loss:  0.7950658635014579
Average_Accuracy:  0.7485907553551296
Confusion Matrix: 
 [[549 153 104]
 [120 682 121]
 [ 62 109 761]]
Epoch:  17 

Validation 

Validation Loss:  1.0705550263325374
Validation Accuracy:  0.4386422976501306
Confusion Matrix: 
 [[ 67  61 171]
 [ 39 107 135]
 [ 12  12 162]]


Batch:  40 / 84
Batch Recognition loss:  0.7546444535255432


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/envs/torch/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-ecaa81700cf1>", line 8, in <module>
    methods.frame_train("global_image_train_v2",num_epochs,model_name,model,train_dataloader,valid_dataloader,optimizer,criterion,device)
  File "/opt/notebooks/Github/EmotiW2020/train/methods.py", line 144, in frame_train
    for i_batch, (frame_batch,_,_,label) in tqdm(enumerate(train_dataloader)):
  File "/opt/conda/envs/torch/lib/python3.7/site-packages/tqdm/notebook.py", line 215, in __iter__
    for obj in super(tqdm_notebook, self).__iter__(*args, **kwargs):
  File "/opt/conda/envs/torch/lib/python3.7/site-packages/tqdm/std.py", line 1097, in __iter__
    for obj in iterable:
  File "/opt/conda/envs/torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 346, in __next__
    data = self._dataset_fetche

KeyboardInterrupt: 